In [16]:
!pip install mpi4py
!apt-get update
!apt-get install -y openmpi-bin libopenmpi-dev

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.7 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.8 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,711 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,677 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://ppa

In [ ]:
%%writefile ola_mpi.py
from mpi4py import MPI

# Inicializa o comunicador
comm = MPI.COMM_WORLD

rank = comm.Get_rank()   # Quem sou eu?
size = comm.Get_size()   # Quantos processos existem?

print(f"Olá! Sou o processo {rank} de um total de {size}", flush=True)

Writing ola_mpi.py


In [ ]:
!mpiexec --allow-run-as-root --oversubscribe -n 4 python -u ola_mpi.py

Olá! Sou o processo 2 de um total de 4
Olá! Sou o processo 1 de um total de 4
Olá! Sou o processo 3 de um total de 4
Olá! Sou o processo 0 de um total de 4


In [ ]:
%%writefile cria_meu_arquivo.py

from mpi4py import MPI             # FIXO: importa o MPI

comm = MPI.COMM_WORLD              # FIXO: comunicador global
rank = comm.Get_rank()             # FIXO: id do processo
size = comm.Get_size()             # FIXO: número total de processos

Writing cria_meu_arquivo.py


In [ ]:
!mpiexec --allow-run-as-root --oversubscribe -n 4 python -u cria_meu_arquivo.py

In [41]:
%%writefile questao1.py

#Crie um sistema de cálculo distribuído simples com apenas 2 processos

from mpi4py import MPI             # FIXO: importa o MPI

comm = MPI.COMM_WORLD              # FIXO: comunicador global
rank = comm.Get_rank()             # FIXO: id do processo
size = comm.Get_size()             # FIXO: número total de processos

# Rank 0 = Cliente
if rank == 0:
  numero = 80
  print(f"O cliente enviou o número: {numero}", flush=True)
  # Envia o número para o servidor (rank 1)
  comm.send(numero, dest=1)

  # Espera a resposta do servidor (rank 1)
  resposta = comm.recv(source=1)

  print(f"O servidor me respondeu: {resposta}", flush=True)

# Rank 1 = Servidor
if rank == 1:
  # Recebe o número do cliente (rank 0)
  numero_recebido = comm.recv(source=0)

  # Processa o número
  resultado = int(numero_recebido / 2)

  # Envia o resultado de volta para o cliente (rank 0)
  comm.send(resultado, dest=0)

Writing questao1.py


In [42]:
!mpiexec --allow-run-as-root --oversubscribe -n 2 python -u questao1.py

O cliente enviou o número: 80
O servidor me respondeu: 40


In [ ]:
# Rank 0 (Cliente)
#   └─ send(80) ─────────► Rank 1 (Servidor)
#                           └─ divide por 2
# Rank 0  ◄──────── recv(40) ── send(40) ─┘

In [43]:
%%writefile questao2.py

#Four-Way Handshake em MPI (send / recv)

from mpi4py import MPI             # FIXO: importa o MPI

comm = MPI.COMM_WORLD              # FIXO: comunicador global
rank = comm.Get_rank()             # FIXO: id do processo
size = comm.Get_size()             # FIXO: número total de processos

# Garantia de segurança, verificando se tem apenas dois 2 processos
if size != 2:
  print("ERRO: execute com exatamente 2 processos", flush=True)
  exit()

# Rank 0 -> Operador
if rank == 0:
  # PASSO 1: Solicitação
  msg1 = "SOLICITAÇÃO DE COMUNICAÇÃO"
  comm.send(msg1, dest=1)
  print(f"[PASSO 1] Operador ENVIOU: {msg1}", flush=True)

  # PASSO 2: Confirmação
  msg2 = comm.recv(source=1)
  print(f"[PASSO 2] Operador RECEBEU: {msg2}", flush=True)

  # PASSO 3: Código de lançamento
  codigo = "CODIGO-XXX"
  comm.send(codigo, dest=1)
  print(f"[PASSO 3] Operador ENVIOU: {codigo}", flush=True)

  # PASSO 4: ACK final
  msg4 = comm.recv(source=1)
  print(f"[PASSO 4] Operador RECEBEU: {msg4}", flush=True)

# Rank 1 -> Satélite
if rank == 1:
  # PASSO 1: Recebe solicitação
  msg1 = comm.recv(source=0)
  print(f"[PASSO 1] Satélite RECEBEU: {msg1}", flush=True)

  # PASSO 2: Envia confirmação
  msg2 = "CONFIRMAÇÃO DE COMUNICAÇÃO"
  comm.send(msg2, dest=0)
  print(f"[PASSO 2] Satélite ENVIOU: {msg2}", flush=True)

  # PASSO 3: Recebe código
  codigo = comm.recv(source=0)
  print(f"[PASSO 3] Satélite RECEBEU: {codigo}", flush=True)

  # PASSO 4: ACK final
  msg4 = "ACK FINAL - CÓDIGO RECEBIDO COM SUCESSO"
  comm.send(msg4, dest=0)
  print(f"[PASSO 4] Satélite ENVIOU: {msg4}", flush=True)

Writing questao2.py


In [44]:
!mpiexec --allow-run-as-root --oversubscribe -n 2 python -u questao2.py

[PASSO 1] Operador ENVIOU: SOLICITAÇÃO DE COMUNICAÇÃO
[PASSO 1] Satélite RECEBEU: SOLICITAÇÃO DE COMUNICAÇÃO
[PASSO 2] Satélite ENVIOU: CONFIRMAÇÃO DE COMUNICAÇÃO
[PASSO 2] Operador RECEBEU: CONFIRMAÇÃO DE COMUNICAÇÃO
[PASSO 3] Operador ENVIOU: CODIGO-XXX[PASSO 3] Satélite RECEBEU: CODIGO-XXX

[PASSO 4] Satélite ENVIOU: ACK FINAL - CÓDIGO RECEBIDO COM SUCESSO
[PASSO 4] Operador RECEBEU: ACK FINAL - CÓDIGO RECEBIDO COM SUCESSO


In [45]:
%%writefile questao3.py

#Escolha do rei vendo apenas vizinhos

from mpi4py import MPI             # FIXO: importa o MPI

comm = MPI.COMM_WORLD              # FIXO: comunicador global
rank = comm.Get_rank()             # FIXO: id do processo
size = comm.Get_size()             # FIXO: número total de processos

# INFRAESTRUTURA FIXA (CABO DE REDE), achei genial
vizinho_direita = (rank + 1) % size
vizinho_esquerda = (rank - 1 + size) % size

# Algoritmo de Chang–Roberts
ativo = True
nao_eleito = True
rei = None

#inicia elição
comm.send(("ELEICAO", rank), dest=vizinho_direita)
print(f"Processo {rank} entrou na eleição enviando seu rank", flush=True)

while True:
  tipo, valor = comm.recv(source=vizinho_esquerda)

  # ELEIÇÃO
  if tipo == "ELEICAO":
    if valor > rank:
      # repassa ID maior
      comm.send(("ELEICAO", valor), dest=vizinho_direita)
      print(f"Processo {rank} repassou o candidato {valor} para o {vizinho_direita}", flush=True)

    elif valor < rank and ativo:
      # descarta e envia o próprio ID
      comm.send(("ELEICAO", rank), dest=vizinho_direita)
      print(f"Processo {rank} descartou o candidato {valor} e enviou se colocou como candidato({rank})", flush=True)
      ativo = False

    elif valor == rank and nao_eleito:
      # deu a volta completa e foi eleito
      rei = rank
      print(f"Processo {rank} foi eleito REI!", flush=True)
      comm.send(("REI", rank), dest=vizinho_direita)
      nao_eleito = False

  # ANÚNCIO DO REI
  if tipo == "REI":
    rei = valor
    if rei != rank:
      print(f"Processo {rank} reconhece como REI o processo {rei}", flush=True)
      comm.send(("REI", rei), dest=vizinho_direita)
    break

Overwriting questao3.py


In [46]:
!mpiexec --allow-run-as-root --oversubscribe -n 10 python -u questao3.py

Processo 8 entrou na eleição enviando seu rank
Processo 4 entrou na eleição enviando seu rank
Processo 0 entrou na eleição enviando seu rankProcesso 6 entrou na eleição enviando seu rank
Processo 3 entrou na eleição enviando seu rank
Processo 4 descartou o candidato 3 e enviou se colocou como candidato(4)

Processo 7 entrou na eleição enviando seu rank
Processo 7 descartou o candidato 6 e enviou se colocou como candidato(7)
Processo 8 descartou o candidato 7 e enviou se colocou como candidato(8)
Processo 2 entrou na eleição enviando seu rank
Processo 9 entrou na eleição enviando seu rank
Processo 9 descartou o candidato 8 e enviou se colocou como candidato(9)
Processo 0 repassou o candidato 9 para o 1
Processo 0 repassou o candidato 9 para o 1
Processo 5 entrou na eleição enviando seu rank
Processo 5 descartou o candidato 4 e enviou se colocou como candidato(5)
Processo 1 entrou na eleição enviando seu rankProcesso 6 descartou o candidato 5 e enviou se colocou como candidato(6)
Process

In [72]:
%%writefile questao4.py

#Escolha de quem vai usar a impresora por concenso

from mpi4py import MPI             # FIXO: importa o MPI

comm = MPI.COMM_WORLD              # FIXO: comunicador global
rank = comm.Get_rank()             # FIXO: id do processo
size = comm.Get_size()             # FIXO: número total de processos

""" Uma impressora com servidor central
#impressora
if rank == 0:
  while True:
    #variavel(is) = esperando(de alguem)
    usuario, imprime = comm.recv()
    print(f"Usuário {usuario} imprimiu: {imprime}")

#usuarios
if rank != 0:
  #envia(envio, destino) | #envia((envio1, envio2, ...), destino)
  comm.send((rank, "A"+str(rank)), dest=0)
"""

#tentei sem fila e não dava certo, ficava parado
fila_de_pedidos = []

quer_imprimir = True        # quer usar a impressora
imprimindo = False          # está usando a impressora?
permissoes = 0              # quantos permissões já recebi

# pedindo permissão a todos
for p in range(size):
  if p != rank:
    comm.send(("REQ", rank), dest=p)

# protocolo distribuído
while True:
  tipo,remetente = comm.recv()

  # RECEBI UM PEDIDO
  if tipo == "REQ":
    # Tenho prioridade? então seguro a permissão
    if imprimindo or (quer_imprimir and rank < remetente):
      fila_de_pedidos.append(remetente)
    else:
      comm.send(("OK", rank), dest=remetente)

  # RECEBI UMA PERMISSÃO
  if tipo == "OK":
    permissoes += 1

    # Se todos liberaram, posso imprimir
    if permissoes == size - 1:

      imprimindo = True
      print(f"Processo {rank} IMPRIMINDO", flush=True)
      imprimindo = False
      print(f"Processo {rank} TERMINOU", flush=True)

      quer_imprimir = False
      permissoes = 0

      #o problema era que aqui ele não enviava para os otros que tinha pedido
      for n in fila_de_pedidos:
        comm.send(("OK", rank), dest=n)
      fila_de_pedidos = []

      break

Overwriting questao4.py


In [73]:
!mpiexec --allow-run-as-root --oversubscribe -n 20 python -u questao4.py

Processo 0 IMPRIMINDO
Processo 0 TERMINOU
Processo 1 IMPRIMINDO
Processo 1 TERMINOU
Processo 2 IMPRIMINDO
Processo 2 TERMINOU
Processo 3 IMPRIMINDO
Processo 3 TERMINOU
Processo 4 IMPRIMINDO
Processo 4 TERMINOU
Processo 5 IMPRIMINDO
Processo 5 TERMINOU
Processo 6 IMPRIMINDO
Processo 6 TERMINOU
Processo 7 IMPRIMINDO
Processo 7 TERMINOU
Processo 8 IMPRIMINDO
Processo 8 TERMINOU
Processo 9 IMPRIMINDO
Processo 9 TERMINOU
Processo 10 IMPRIMINDO
Processo 10 TERMINOU
Processo 11 IMPRIMINDO
Processo 11 TERMINOU
Processo 12 IMPRIMINDO
Processo 12 TERMINOU
Processo 13 IMPRIMINDO
Processo 13 TERMINOU
Processo 14 IMPRIMINDO
Processo 14 TERMINOU
Processo 15 IMPRIMINDO
Processo 15 TERMINOU
Processo 16 IMPRIMINDO
Processo 16 TERMINOU
Processo 17 IMPRIMINDO
Processo 17 TERMINOU
Processo 18 IMPRIMINDO
Processo 18 TERMINOU
Processo 19 IMPRIMINDO
Processo 19 TERMINOU


In [24]:
%%writefile questao5.py

#General Romano Leal ou Traidor

from mpi4py import MPI
import random

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Lista de ordens possíveis
opcoes = ["ATACAR", "RECUAR"]

# GENERAL (rank 0)
if rank == 0:
    # 1. Decide a personalidade do General (50% de chance de ser traidor ou e 50% de não ser)
    sou_traidor = random.choice([True, False])

    if sou_traidor:
      print("[General] Decidi ser um TRAIDOR ! Vou confundiros tenentes .", flush=True)
      for i in range(1, size):
        # General Traidor : Envia ordens aleatórias para cada um
        ordem_falsa = random.choice(opcoes)
        print(f"[General] -> Enviando '{ordem_falsa}' para Tenente {i}", flush=True)
        comm.send(ordem_falsa, dest=i)
    else:
      # General Leal : Escolhe uma ordem e manda a MESMA para todos
      ordem_real = random.choice(opcoes)
      print(f"[General] Sou LEAL. Ordem única: {ordem_real}", flush=True)
      for i in range(1, size):
        comm.send(ordem_real, dest=i)

# TENENTES (rank >= 1)
else:
    # recebe ordem do General
    ordem_recebida = comm.recv(source=0)
    print(f"[Tenente {rank}] Recebi do General a ordem: {ordem_recebida}", flush=True)

    # envia ordem para os outros tenentes
    for i in range(1, size):
      if i != rank:
        comm.send(ordem_recebida, dest=i)

    # recebe ordens dos outros tenentes
    ordens = [ordem_recebida] #cria a lista das ordens para comparar e armazena a sua ordem
    for _ in range(size - 2):
      ordem = comm.recv()
      ordens.append(ordem)

    # decisão por maioria
    decisao = max(set(ordens), key=ordens.count)
    print(f"[Tenente {rank}] Ordens recebidas: {ordens}", flush=True)
    print(f"[Tenente {rank}] DECISÃO FINAL: {decisao}", flush=True)

    # verificar se o general é traidor ou leal
    general_leal = True

    for n in range(1, len(ordens)):
      if ordens[0] != ordens[n]:
        general_leal = False
        break

    if general_leal:
      print(f"[Tenente {rank}] General é LEAL", flush=True)
    else:
      print(f"[Tenente {rank}] General é TRAIDOR", flush=True)

Overwriting questao5.py


In [25]:
!mpiexec --allow-run-as-root --oversubscribe -n 11 python -u questao5.py

[Tenente 6] Recebi do General a ordem: ATACAR
[Tenente 7] Recebi do General a ordem: RECUAR
[Tenente 8] Recebi do General a ordem: ATACAR
[General] Decidi ser um TRAIDOR ! Vou confundiros tenentes .
[General] -> Enviando 'RECUAR' para Tenente 1
[General] -> Enviando 'ATACAR' para Tenente 2
[General] -> Enviando 'ATACAR' para Tenente 3
[General] -> Enviando 'RECUAR' para Tenente 4
[General] -> Enviando 'RECUAR' para Tenente 5
[General] -> Enviando 'ATACAR' para Tenente 6
[General] -> Enviando 'RECUAR' para Tenente 7
[General] -> Enviando 'ATACAR' para Tenente 8
[General] -> Enviando 'ATACAR' para Tenente 9
[General] -> Enviando 'ATACAR' para Tenente 10
[Tenente 1] Recebi do General a ordem: RECUAR
[Tenente 5] Recebi do General a ordem: RECUAR
[Tenente 9] Recebi do General a ordem: ATACAR
[Tenente 10] Recebi do General a ordem: ATACAR
[Tenente 3] Recebi do General a ordem: ATACAR
[Tenente 4] Recebi do General a ordem: RECUAR
[Tenente 2] Recebi do General a ordem: ATACAR
[Tenente 1] Orden